# 🤖 Microsoft Agent Framework

## ⚠️ Why Not Autogen?

**Autogen is now a dead end for the future.** (September 2025)

- Microsoft officially put **Autogen into maintenance mode** — no new features will be added.
- All new development, innovation, features, and support go to **Agent Framework**.
- Enterprises are migrating to Agent Framework.
- If you learn Autogen deeply now, you’ll be learning a deprecated ecosystem.

## 📚 Main Concepts

**Microsoft Agent Framework** consists of these core components:

- **🤖 Agents** - AI entities that can reason, use tools, and communicate
- **🔄 Workflows** - Data-flow graphs that orchestrate agents and tasks with explicit control
- **💬 Messages** - How agents communicate (typed and routed through specific paths)
- **🔌 Model Clients** - Connect to LLMs (OpenAI, Azure, etc.)
- **🛠️ Tools** - Functions agents can call (web search, code execution, APIs)
- **💾 State Management** - Thread-based system for conversation history and context
- **💾 Checkpointing** - Save and resume long-running workflows

Microsoft Agent Framework's LLM calls are **asynchronous** (non-blocking), so they use async/await to allow your program to do other things while waiting for the LLM response instead of freezing.

---
📢 Discover more Agentic AI notebooks on my [GitHub repository](https://github.com/lisekarimi/agentverse) and explore additional AI projects on my [portfolio](https://lisekarimi.com).

In [ ]:
# uv add agent-framework --pre

In [ ]:
import asyncio
import gradio as gr
import os
from dotenv import load_dotenv
from agent_framework.openai import OpenAIChatClient
from typing import Annotated
import requests
from agent_framework import (ChatAgent, ChatMessageStore)
from pydantic import Field

In [ ]:
load_dotenv(override=True)
MODEL_ID = "gpt-4o-mini"
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")

# Validate required environment variables
if not pushover_user or not pushover_token:
    print("Warning: PUSHOVER_USER and PUSHOVER_TOKEN are required. Please set them in your .env file.")

In [ ]:
# Define the Pushover tool
def send_pushover_notification(
    message: Annotated[str, Field(description="The message to send")],
    title: Annotated[str, Field(description="The notification title")] = "Agent Alert"
) -> str:
    """Send a push notification via Pushover."""

    response = requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": pushover_token,
            "user": pushover_user,
            "message": message,
            "title": title
        }
    )

    return "✅ Notification sent" if response.status_code == 200 else "❌ Failed"

## Simple Agent with tool

In [ ]:
async def main():
    agent = ChatAgent(
        chat_client=OpenAIChatClient(model_id=MODEL_ID),
        instructions="You are a helpful assistant that can send push notifications via Pushover.",
        tools=[send_pushover_notification]
    )

    print("=== Test 1: Simple Question ===")
    result = await agent.run("What is 5 + 3?")
    print(result.text)
    print()

    print("=== Test 2: Send Notification ===")
    result = await agent.run("Send me a notification saying 'Hello from Microsoft Agent Framework!'")
    print(result.text)

# For Jupyter
await main()

## Agent with tool and memory

In [ ]:
# Create message store for memory
message_store = ChatMessageStore()

# Create agent with memory (thread-based)
agent = ChatAgent(
    chat_client=OpenAIChatClient(model_id=MODEL_ID),
    instructions="You are a helpful assistant. Use the pushover tool when user asks to send notifications.",
    tools=[send_pushover_notification],
    chat_message_store_factory=lambda: message_store
)

# Chat function with memory
async def chat_async(message, history):
    # Run agent with thread_id for memory
    result = await agent.run(message, thread_id="user_1")
    return result.text

# Wrapper for Gradio (Gradio doesn't support async directly)
def chat(message, history):
    return asyncio.run(chat_async(message, history))

# Launch Gradio
demo = gr.ChatInterface(
    chat,
    title="Microsoft Agent Framework Chatbot",
    description="Chat with memory + Pushover notifications",
)

demo.launch()